In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from itertools import combinations
import string
from collections import Counter, defaultdict
import re

In [2]:
path_connected_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\"
path_temp_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\temporaryData\\"
year = "2009"

full_path = f"{path_connected_data}{year}_gptDataframe.csv"
split_path = f"{path_connected_data}{year}_split_positions.csv"
vp_path = f"{path_temp_data}{year}_vice_presidents.csv"
grouped_path = f"{path_temp_data}{year}_grouped.csv"

pd.options.display.max_rows = 400

In [3]:
#define keywords
keywords = {
    "Arts and Sciences": ["arts and science", "letters and science", "arts, sciences, and letters", "arts, sciences", "arts & sciences", "letters, sciences", "arts and letters", "arts & letters"],
    "Engineering": ["engineering", "mines", "aerospace", "aeronautics", "polytechnic studies", "aviation", "applied science", "computational science"],
    
    "Computer Science": ["computer science", "computer,", "software engineering", "and computer", "computing"],
    "Data Science": ["data science", "artificial intelligence", "machine learning"],
    "Technology": ["technology"],
    "Information": ["information science", "information management", "information systems", "information stud", "library science", "and information", "library stud", "informatics", "school of information", 
                    "information and", "information school"],
    "Archictecure": ["architecture", "architectural", "urban design", "urban planning", "environmental design", "environment and design", "design, construction", "school of construction"],
    

    "Behavioral Science": ["psychology", "psychological", "human science", "human ecology", "behavioral", "human and", "childhood stud"],
    "Social Sciences": ["social science", "social and", "social ecology"],
    "Natural Sciences": ['natural science', "mathematics", "chemistry", "biology", "mathematical", "physical science", "life science", "biological science", 
                        "biochemistry", "natural and", "basic", "college of science", "school of science", ", science, and", ", science and", "physics", "college of the sciences"],
    "Environmental and Earth Science": ["environmental science", "forestry", "natural resource", "college of forest", 
                              "school of forest", "school of environment ", "environmental and", "geology", "oceanography", "earth science", "geophysics", "seismology", "volcanology", "earth and" 
                              "paleontology", "mineral science", "geoscience", "earth science", "earth studies", "atmospheric", "college of the environment", "school of ecology", "school of environment"],
    "Marine Science": ["marine science", "marine stud", "ocean science", "oceanography", "oceanographic", "ocean and"],

    
    "Business": ["business", "commerce", "mba", "college of economics", "and economics", "finance", "entrepreneurship", "school of management", "and management", "accounting", "agribusiness", "economic,", "accountancy", "schools of management", "college of management"],
    "Hospitality": ["hospitality", "hotel administration", "hotel management", "restaurant management"],
    "Education": ["college of education", "school of education", "teaching", "teach", "curriculum development", "human development", "and education", ", education and"],
    "Criminal Justice": ["criminal justice", "criminology", "criminal", "forensics"],


    "Law": ["law", "legal studies"],
    "Policy and Affairs": ["public policy", "international policy", "public affairs", "international affairs", "environmental affairs", "urban affairs", "metropolitan affairs", "international service", "leadership studies", 
                           "diplomacy", "diplomat", "international studies", "public administration", "urban policy", "global studies", "international relations", "social policy", "peace studies", "global management", "policy studies",
                           "policy, planning", "school of leadership", "policy science"],
    

    "Agriculture": ["agriculture", "agricultural", "agribusiness"],
    "Family and Consumer Science": ["family and", "home economics", "consumer science", "family science", "home", "family studies"],
    "Culinary": ["culinary", "cooking", "gastronomy"],


    "Journalism": ["journalism"],
    "Communication": ["communication"],
    "Liberal Arts": ["liberal art", "liberal studies", "interdisciplinary studies", "general studies", "liberal and", "individualized study"],
    "Humanities": ["humanities", "history", "historical", "philosophy", "literature", "political"],
    "Fine Arts": ["fine arts", "music", "theater", "creative", "theatre", "film", "performing art", "the arts", "design art", "dramatic arts", "dance",
                   "visual arts", "school of drama", "recording arts", "ceramics", "fine and applied art", "school of design", "art and design", "fine", 
                   "cinema", "art ", "art, ", "studio art", "media art", "arts and performance", "drama", "arts and media", "textiles", "applied arts"],
    "Language and Culture": ["languages", "lingusitics", "cultures", "translational studies", "intercultural studies", "ethnic studies", "hebraic studies", "cultural stud", "asian studies", "culture ", " hawaiian knowledge", 
                             "jewish", "asian and pacific studies"],


    "Nursing": ["nursing"],
    "Physical Therapy and Nutrition": ["physical therapy", "nutrition", "physiotherapy", "nutritional therapy", "rehab", "human performance"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine", "human medicine"],
    "Pharmacy": ["pharmacy", "pharmacology"],
    "Dentistry" : ["dentistry", "dental", "denistry"],
    "Optometry" : ["optometry", "optics", "optical science"],
    "Veterinary" : ['veterinary'],
    "Audiology": ["audiology"],
    "Gerontology": ["gerontology"],
    "Pathology": ["pathology"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome", "occupational therapy"],


    "Religious Studies": ["religious", "theology", "divinity", "biblical", "theological", "religion", "seminary", "religous", "christ college", "talmudic", "torah", "quran", "bible"],
    "Social Work": ["social work", "social welfare", "community service", "human service", "social service", "public service", "social administration"],
    

    "Graduate": ["graduate school", "graduate studies", "graduate college", "college of graduate", "school of graduate",  "advancing", "advanced study", "graduate division", "graduate and professional studies", "doctoral", "graduate study", "graduate "],
    "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning", "adult learning",
                           "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "distance learning", "continuous education", "extended university"],
    "Professional Studies": ["professional studies", "professional development", "professional education", "school of professions", "professional campus"],
    # "Administration": ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
    #                    "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", 
    #                    "university librar", "information service", "information access", "library access", "residence", "academic affairs", "library affairs", "academic succes", "first year student",
    #                    "undergraduate studies", "undergraduate education"],
    "Honor College": ["honor college", "honors college", "scholar program"],

    # "Satellite Campus": ["campus"],
    # "Foundation": ["foundation"],
    # "Boards": ["board of", "regents", "trustees", "council of"],
    "Hospital": ["hospital"],
    "University College": ["university college"],
    "Error": []
}

In [4]:
VP_KEYWORDS = {
    # "Head VP": ["head vice president"],
    "SubInstitution": ["arts and science", "the college", "the arts", "interdisciplinary studies", "life sciences", "agriculture", "agricultural", "liberal arts", "engineering", "law center"],
    "Generic": [],
    "Foundation": ["foundation"],


    "Administration": ["vice president, administration", "and administration", "administrative", "audit and compliance", "legal affairs", "legal services", "institutional affairs", "university affairs", "vice president, operations", 
                       "employment service", "vice president, quality", "management service", "university compliance" "university operation", "institutional services", "general administration",
                       "of the corporation", "special program", "university governance", "materials management", "executive affair"," university operation", "vice president. administration"],
    "Boards": ["of the board", "board of", "board operation"],
    "Campus Operations": ["sustainability", "facilities", "campus operations", "infrastructure", "environmental", "security", "campus police", "emergency management", "safety", 
                          "campus services", "university services", "facility operation", "university architect", "land, buildings and real estate", "auxiliary", "capital construction",
                          "property management", "plant operations", "construction", "campus residence", "physical resources", "university compliance", "physical plant", "facilities"],

    "Health Affairs": ["health service", "health affair", "ambulatory", "clinical service", "cancer", "patient care", "diagnostic service", "physicians",
               "clinical", "care contracting", "outpatient", "patient service", "medical affair", "health system affairs", "surgical service", "health system",
               "community health", "health policy"],


    "Academic Affairs": ["undergraduate education", "curriculum", "provost" "undergraduate studies", "graduate studies", "instruction", "academic program", "academic development",
                         "education affair", "undergraduate affair", "academie affairs", "educational and", "teacher education", "experiential education", "academic affair", "academic administration",
                         "undergraduate initiative", "cooperative education", "school program", "interdisciplinary education", "educational outreach", "educational,", "undergraduate studies", "educational affair",
                         "academic service", "academic afair", "educational resource", "academic enrichment", "academic resource", "university studies", "vice president, academics", "academic operation",
                         "educational development", "academic issues", "academic planning", "career development"],
    "Faculty Affairs": ["faculty affair", "staff relations", "vice president, faculty"],
    "Research": ["innovation", "research", "grants", "publication", "biotechnology", "scientific", "sponsored programs"],
    

    "Student Affairs": ["student", "student life", "student service", "support", "wellbeing", "well-being", "campus life", "housing", "counseling", "student affairs", "student initiative", "student service", 
                        "student success", "student activities", "student development", "students", "university life", "student learning", "residence life", "student and community",
                        "career service", "student judicial service", "advisement", "student access", "study abroad", "dining program", "academic life"],
    "Enrollment": ["enrollment", "financial aid", "retention", "registrar", "admission", "enrolment"],
    

    "Information Service": ["technology", "information", "librar", "data management", "computer", "computing", "network service", "internet service", "electronic marketing", "converging technologies",
                    "technical service", "vice president, systems", "enterprise system", "records", "system administration"],


    "DEI": ["diversity", "equity", "inclusion", "affirmative action", "dei", "multicultural affairs", "equal opportunity", "minority affair", "intercultural affair"],
    "External Relations": [ "external", "university relation", "college relation", "public relation", "communications", "public affair", "media", "outreach", "partnership", "global",
                            "international", "study abroad", "federal relations", "public events", "institutional relation",
                           "government", "state relations", "community engagement", "community affair", "civic engagement", "community relation", "corporate relation", 
                           "foundation relation", "legislative", "policy affairs", "defense relation", "constituent relations", "regional affair", "system relation", "regional operation",
                           "federal program", "agency relation", "regional engagement", "communication", "news services", "federal affair", "commonwealth relation", "state affair", "institute relation", "community services",
                           "intercampus", "urban affairs", "public service", "school relation", "public policy", "district of columbia affair", "d.c. affair", "vice president, engagement", "vice president, policy", "institute affair",
                           "community development"],


    "Planning": ["planning", "assessment", "evaluation", "planner", "special projects", "strategy and measurement", "institutional effectiveness", "presidential initiative", "capital improvement", "strategy and policy",
                 "university projects", "institutional analysis", "commercialization strategies", "procurement", "strategic affair", "university initiative", "program development", "strategic initiative", "strategic development",
                 "strategic and"],
    "Finance": ["budget", "finance", "financial", "accounting", "marketing", "endowment",  "investment", "business", "fiscal", "and resource management", "institutional resources", "treasury management", "university resources", "audit", ", resource management",
                "commerce", "asset management"],
    "Advancement": ["fundraise", "fundraising", "alumni", "donation", "parent relation", "annual giving", "special events", "principal gifts", "major gift", "campaigns", 
                    "planned giving", "leadership gifts", "gift program", "trusts and estate", "giving", "philanthropy", "constituent program", "institutional advancement", "vice president, advancement", "vice president, development",
                    "university advancement", "university development", "vice president. development", "vice president. advancement", "economic advancement", "economic development",
                    "college advancement", "institute advancement", "and development", "and advancement", "resource development", "institutional advancement", "sponsored funds", "college development", "wide advancement", "institution advancement",
                    "regional development", "planned gift"],
    "Human Resources": ["hr", "human resource", "personnel", "recruitment", "employee relations", "union", "collective bargaining", "labor", "human relations", "human capital", "personnel", "human and", "professional development", "human service", "hiring",
                        "workforce"],
    

    "Religious": ["ministry", "mission", "ministries", "church", "religious", "religion", "spiritual", "christian development"],
    "Satellite Campus": ["satellite campus"],
    "Athletics": ["athletic", "sports"],

    "Other": []
}

In [5]:
#Extract the names of all the institutions for validation
def extract_institutions(df):
    institution_list = []
    for index, row in df.iterrows():
        if row["Institution"] not in institution_list:
            institution_list.append(row["Institution"])
    return institution_list

In [6]:
def extract_all_vp(df):
    vp_df = []
    for index, row in df.iterrows():
        position = row["Position"].title()
        if "Vice President" in position.title():
            vp_df.append(row)
    return pd.DataFrame(vp_df)

def check_keyword(row):
    blank = False
    position = row["Position"].title()
    institution = row["Institution"].title()
    subinstitution = row["SubInstitution"]
    if pd.isna(subinstitution):
        blank = True

    classification = ""
    for key, value in VP_KEYWORDS.items():
        value_title = [s.title() for s in value]
        vp_index = position.lower().find("vice president")
        vp_type = position[vp_index + len("vice president"):].strip()
        contains_keyword = any(word in position for word in value_title)
        if contains_keyword:
            classification = classification + key + ", "
        elif position == "Vice President" or position == "Executive Vice President" or position == "Head Vice President" or position == "Chief Vice President":
            if blank:
                classification += "Generic"
                break
            else:
                classification += "SubInstitution"
                break
        elif blank and ((vp_type == "") or ("," not in position and "." not in position)):
            print(position, "     " , subinstitution)
            classification += "Generic"
            break
        elif institution in position and "Foundation" not in position:
            classification += "Satellite Campus"
            break
    return classification.rstrip(", ")

def classify_vp(vp_df):
    classified_df = vp_df.copy()
    classified_df['FixedPosition'] = ""
    for index, row in vp_df.iterrows():
        vp_type = check_keyword(row)
        classified_df.at[index, 'FixedPosition'] = vp_type
    return classified_df

def remaining_classification(vp_df):
    for index, row in vp_df.iterrows():
        type = row['FixedPosition']
        type_blank = False
        if type == "":
            type_blank = True
        position = row["Position"].title()
        if "Campus" in position and type_blank == True:
            vp_df.at[index, 'FixedPosition'] += "Satellite Campus, "

        if "Academic" in position and type_blank == True:
            vp_df.at[index,'FixedPosition'] += "Academic Affairs, "
        if "Administration" in position and type_blank == True:
            vp_df.at[index, 'FixedPosition'] += "Administration, "
        if ("Development" in position or "Advancement" in position) and type_blank == True:
            vp_df.at[index, 'FixedPosition'] += "Advancement, "
        if type == "" and "Campus" in position:
            vp_df.at[index, 'FixedPosition'] = "Satellite Campus"
        elif type == "":
            vp_df.at[index, 'FixedPosition'] = "Other"
        vp_df.at[index, 'FixedPosition'].rstrip(", ")
    return vp_df
        
def assign_position_grouping_subinst(position):
    classification = ""
    for key, value in keywords.items():
        value_upper = [s.title() for s in value]
        keyword_appears = any(s in position for s in value_upper)
        dean_comma = position.rfind("Dean,") + 5
        if key.title() == position[dean_comma:]:
            classification += key + ", "
        elif keyword_appears:
            classification += key + ", "
    return classification.strip(", ")

def classify_subinstitution_vp(vp_df, grouped_df):
    subinstitution_dict = grouped_df.set_index(['Institution', 'SubInstitution'])['Category'].to_dict()
    for index, row in vp_df.iterrows():
        position = row["Position"].title()
        key = (row["Institution"], row["SubInstitution"])
        vp_index = row["Position"].rfind("Vice President") + 14
        if ("SubInstitution" in row['FixedPosition'] or "Other" in row['FixedPosition']) and (not pd.isna(row["SubInstitution"]) and vp_index == len(position)):
                if subinstitution_dict[key] != "Error":
                    vp_df.at[index, 'FixedPosition'] = subinstitution_dict[key]
                else:
                    vp_df.at[index, 'FixedPosition'] = "Generic"
                # print(key, "     ", subinstitution_dict[key])
        elif "SubInstitution" in row['FixedPosition'] and vp_index != len(position):
            classification = assign_position_grouping_subinst(position)
            vp_df.at[index, 'FixedPosition'] = classification
        elif vp_index != len(position) and pd.isna(row["SubInstitution"]) and "Other" in row['FixedPosition']:
            classification = assign_position_grouping_subinst(position)
            if classification != "":
                vp_df.at[index, 'FixedPosition'] = classification
            else:
                vp_df.at[index, 'FixedPosition'] = "Other"
        if vp_df.at[index, 'FixedPosition'] == "Other":
            classification = assign_position_grouping_subinst(position)
            if classification != "":
                vp_df.at[index, 'FixedPosition'] = classification
            else:
                vp_df.at[index, 'FixedPosition'] = "Other"
    return vp_df
    

def last_sweep(vp_df):
    for index, row in vp_df.iterrows():
        if pd.isna(row['FixedPosition']) or row['FixedPosition'] == "":
            vp_df.at[index,'FixedPosition'] = "Other"
    return vp_df
                



In [7]:
full_df = pd.read_csv(split_path)
grouped_df = pd.read_csv(grouped_path)

In [8]:
vp_df = extract_all_vp(full_df)
classified_vp_df = classify_vp(vp_df)
classified_vp_df = remaining_classification(classified_vp_df)
classified_vp_df = classify_subinstitution_vp(classified_vp_df, grouped_df)
classified_vp_df = last_sweep(classified_vp_df)
# classified_vp_df.to_csv(vp_path, index = False)

Senior Vice President       nan
Deputy Vice President       nan
Senior Vice President       nan
Associate Vice President       nan
Associate Vice President       nan
Senior Vice President       nan
Senior Vice President       nan
Senior Vice President       nan
Associate Vice President       nan
Senior Vice President       nan
Associate Vice President       nan
Senior Vice President       nan
Associate Vice President       nan
Associate Vice President       nan
Associate Vice President       nan
Senior Vice President       nan
Senior Vice President       nan
Academic Vice President       nan
Administrative Vice President       nan
Advancement Vice President       nan
Information Technology Vice President       nan
International Vice President       nan
Student Life Vice President       nan
Associate International Vice President       nan
Associate Student Life Vice President       nan
Assistant Information Technology Vice President       nan
Assistant International Vice President      

In [9]:
# for index, row in classified_vp_df.iterrows():
#     type = row['FixedPosition']
#     if type == "Other":
#         print(row["Position"])

for index, row in classified_vp_df.iterrows():
    pos = row["Position"]
    type = row['FixedPosition']
    if type[-1] == ',':
        print(pos)

In [10]:
vp_types = []

for index, row in classified_vp_df.iterrows():
    type = row['FixedPosition']
    if type == "Other":
        position = row["Position"]
        vp_index = position.lower().find("vice president")
        if vp_index != -1:
            vp_type = position[vp_index + len("vice president"):].strip()
            vp_types.append(vp_type)

vp_counts = Counter(vp_types)
print(vp_counts)


Counter({', Health': 2, ', Medicine': 2, ', Usf Health': 2, ', Strategic Capital Projects': 1, ', Baylor Network': 1, ', Strategic Enterprises': 1, ', Manhattanville Development': 1, '(Interim)': 1, ', Atlanta': 1, ', Strategic Alliance': 1, ', Extended Programs': 1, ', University Applications': 1, 'Of The College': 1, ', Office Development': 1, ', Community Colleges (Interim)': 1, '': 1, ', Health Operations': 1, ', Scholarly and Cultural Affairs': 1, ', Molecular Medicine and Genetics': 1, ', Prospect Development': 1, ', Rural Health': 1, ', Legal Education': 1})


In [11]:
print(len(classified_vp_df))
count = 0
for index, row in classified_vp_df.iterrows():
    type = row['FixedPosition']
    if type == "Other":
        count+=1
    elif type == "Generic":
        print(row["Position"])
print("Remaining unclassified: ")
print(100*(count/len(classified_vp_df)),"%")

5121
Executive Vice President
Vice President
Vice President
Executive Vice President
Senior Vice President
Vice President
Deputy Vice President
Vice President
Executive Vice President
Vice President
Senior Vice President
Vice President
Executive Vice President
Associate Vice President
Associate Vice President
Executive Vice President
Senior Vice President
Senior Vice President
Senior Vice President
Vice President
Associate Vice President
Vice President
Vice President
Senior Vice President
Associate Vice President
Vice President
Vice President
Executive Vice President
Senior Vice President
Executive Vice President
Vice President
Associate Vice President
Associate Vice President
Associate Vice President
Executive Vice President
Senior Vice President
Senior Vice President
Vice President
Executive Vice President
Academic Vice President
Administrative Vice President
Advancement Vice President
Information Technology Vice President
International Vice President
Student Life Vice President
Asso

In [12]:
#count of each type
type_count = Counter(classified_vp_df['FixedPosition'])
type_dict = dict(sorted(type_count.items(), key=lambda item: item[1], reverse=True))

for key, value in type_dict.items():
    print(key, ": ", value)

Advancement :  654
Generic :  550
Academic Affairs :  462
Student Affairs :  442
External Relations :  441
Finance :  411
Administration, Finance :  241
Information Service :  215
Research :  188
Administration :  172
Enrollment :  161
Human Resources :  149
Campus Operations :  96
Planning :  71
External Relations, Finance :  66
Health Affairs :  62
External Relations, Advancement :  54
Student Affairs, Enrollment :  51
Planning, Finance :  38
DEI :  36
Health Science :  36
Religious :  28
Other :  25
Academic Affairs, Research :  24
Research, Advancement :  18
Satellite Campus :  16
Enrollment, Finance :  16
Campus Operations, Planning :  13
Medical :  13
Faculty Affairs :  12
Research, Planning :  12
Administration, Planning :  10
Enrollment, External Relations :  9
Student Affairs, External Relations :  9
Academic Affairs, Student Affairs :  9
External Relations, Planning :  9
DEI, Human Resources :  8
Academic Affairs, Planning :  8
Agriculture :  8
Administration, Information Ser

In [13]:
for index, row in classified_vp_df.iterrows():
    classified_vp_df.at[index, "FixedPosition"] = "Vice President, " + row["FixedPosition"]

classified_vp_df = classified_vp_df.sort_values(by=["FixedPosition", "Institution"])
classified_vp_df.to_csv(vp_path, index = False)